* Section 1

  - <a href="#ScriptSpt1pt1">Script S.1.1: QuTiP For Exact Solutions</a>

* Section 2

  - <a href="#ScriptSpt2pt1">Script S.2.1: Amplitude Damping Channel</a>

* Section 3

  - <a href="#ScriptSpt3pt1">Script S.3.1: FMO Complex Parameters</a>

  - <a href="#ScriptSpt3pt2">Script S.3.2: FMO Complex Plot</a>

* Section 4

  - <a href="#ScriptSpt4pt1">Script S.4.1: Vectorized Effective Hamiltonian</a>

* Section 5

  - <a href="#ScriptSpt5pt1">Script S.5.1: Defining Pool Of Operators/Gates</a>

* Section 6

  - <a href="#ScriptSpt6pt1">Script S.6.1: Amplitude Damping Channel</a>

* Section 7

  - <a href="#ScriptSpt7pt1">Script S.7.1: Preprocessing Hamiltonian For SSE Approach</a>

* Section 8

  - <a href="#ScriptSpt8pt1">Script S.8.1: Parallel Processing Of Trajectories</a>

* Section 9

  - <a href="#ScriptSpt9pt1">Script S.9.1: Setting Up Parameters For FMO Complex</a>

  - <a href="#ScriptSpt9pt2">Script S.9.2: Setting Up Parameters For Trajectory Method</a>

  - <a href="#ScriptSpt9pt3">Script S.9.3: Plot For FMO Complex Simulation Output</a>

# QFlux Installation

In [ ]:
!pip install qflux

# Section 1

## Script S.1.1: QuTiP For Exact Solutions <a name="ScriptSpt1pt1"></a>

In [ ]:
from qutip import mesolve, Qobj

def qutip_prop(H, rho0, time_arr, c_ops, observable):
    """
    First import the mesolve function, which is used to solve master equations, and the Qobj class, which is used to represent quantum objects, from the QuTiP library.
    - H: Hamiltonian of the system (Qobj).
    - rho0: Initial density matrix (Qobj).
    - time_arr: Time array for dynamic simulation (array).
    - c_ops: List of collapse operators (list of Qobj), can be empty for Liouville equation.
    - observable: Operator for which the expectation value is to be calculated (Qobj).
    Returns:
    - expec_vals: List of expectation values of the observable over time.
    """
    result = mesolve(H, rho0, time_arr, c_ops, e_ops=observable)
    return result.expect


# Section 2

## Script S.2.1: Amplitude Damping Channel <a name="ScriptSpt2pt1"></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

# Pauli matrices and lowering operator
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sp = (sx + 1j * sy) / 2
sm = Qobj(sp)  # collapse operator for amplitude damping

# Identity Hamiltonian (trivial in this case)
H = Qobj(np.eye(2, dtype=np.complex128))

# Time scale
tf = 1000e-12  # final time
dt = 1000e-14  # time step
times = np.arange(0, tf, dt)  # time array

# Amplitude damping rate
gamma = 1.52e9  # damping rate

# Initial state (normalize if necessary)
u0 = np.array([1 / 2, np.sqrt(3) / 2], dtype=np.complex128)
u0 = u0 / np.linalg.norm(u0)  # normalize the state
psi0 = Qobj(u0)  # initial state as Qobj
rho0 = psi0 * psi0.dag()
# Collapse operators
c_ops = [np.sqrt(gamma) * sm]  # amplitude damping

# Define projectors for ground and excited states
proj_excited = basis(2, 1) * basis(2, 1).dag()  # |1><1|
proj_ground = basis(2, 0) * basis(2, 0).dag()   # |0><0|

# Solve the master equation using mesolve
result = qutip_prop(H, rho0, times, c_ops, [proj_ground, proj_excited])

# Extract ground and excited state populations
ground_population = result[0]  # <0|rho|0> (ground state population)
excited_population = result[1]  # <1|rho|1> (excited state population)

# Plotting results
plt.plot(times * 1e12, ground_population, label="Ground State Population |0>")
plt.plot(times * 1e12, excited_population, label="Excited State Population |1>")
plt.xlabel('Time (ps)')
plt.ylabel('Population')
plt.legend()
plt.title('Evolution of Ground and Excited State Populations')
plt.show()


# Section 3

## Script S.3.1: FMO Complex Parameters <a name="ScriptSpt3pt1"></a>

In [ ]:
from qutip import Qobj
import numpy as np
# Hamiltonian (Unit: eV)
H = Qobj([
    [0, 0, 0, 0, 0],
    [0, 0.0267, -0.0129, 0.000632, 0],
    [0, -0.0129, 0.0273, 0.00404, 0],
    [0, 0.000632, 0.00404, 0, 0],
    [0, 0, 0, 0, 0],
])

alpha, beta, gamma = 3e-3, 5e-7, 6.28e-3 #Unit: fs (femtosecond)

# Define the alpha operators
Llist_f = [Qobj(np.diag([0] * i + [np.sqrt(alpha)] + [0] * (4 - i))) for i in range(1, 4)]

# Define the beta operators
Llist_f += [Qobj(np.array([[np.sqrt(beta) if i == 0 and j == k else 0 for j in range(5)] for i in range(5)])) for k in range(1, 4)]

# Define the gamma operator
L_temp = np.zeros((5, 5))
L_temp[4, 3] = np.sqrt(gamma)
Llist_f.append(Qobj(L_temp))


## Script S.3.2: FMO Complex Plot <a name="ScriptSpt3pt2"></a>

In [ ]:
# Measurement operators
Mexp_f = [
    Qobj(np.diag([0, 1, 0, 0, 0])),
    Qobj(np.diag([0, 0, 1, 0, 0])),
    Qobj(np.diag([0, 0, 0, 1, 0])),
    Qobj(np.diag([1, 0, 0, 0, 0])),
    Qobj(np.diag([0, 0, 0, 0, 1]))
]

# Time evolution
times = np.linspace(0.0, 450.0, 2000)
psi0_f = Qobj([[0], [1], [0], [0], [0]])

# Using qutip_propagation function
population = qutip_prop(H, psi0_f, times, Llist_f, Mexp_f)

# Plotting the results
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
labels = ["State 1", "State 2", "State 3", "Ground State", "Sink State"]
for expec, label in zip(population, labels):
    ax.plot(times, expec, label=label)
ax.set_xlabel('Time(fs)')
ax.set_ylabel('Population')
ax.legend()
plt.show()


# Section 4

## Script S.4.1: Vectorized Effective Hamiltonian <a name="ScriptSpt4pt1"></a>

In [ ]:
def vectorize_comm(A):
    # Create an identity matrix with the same dimension as A
    iden = np.eye(A.shape[0])
    # Compute the vectorized commutator [A, .] as the Kronecker product
    return np.kron(iden, A) - np.kron(A.T, iden)

class VectorizedEffectiveHamiltonian_class:
    def __init__(self, He, Ha):
        self.He = He
        self.Ha = Ha

def VectorizedEffectiveHamiltonian(H, gamma, lind):
    # Create an identity matrix with the same dimension as H
    iden = np.eye(H.shape[0])
    d = H.shape[0]
    # Compute the vectorized commutator for the Hamiltonian H
    vec_H = vectorize_comm(H)
    # Initialize the result matrix with zeros (complex type)
    res = np.zeros((d**2, d**2), dtype=np.complex128)
    # Compute the conjugate of the Lindblad operator
    L_conj = lind.conj()
    L_dagger_L = L_conj.T @ lind
    # Compute the Lindblad contribution to the effective Hamiltonian
    res -= gamma * (np.kron(L_conj, lind) - (np.kron(iden, L_dagger_L) + np.kron(L_dagger_L.T, iden)) / 2)
    # Return an instance of the VectorizedEffectiveHamiltonian_class with vec_H and res
    return VectorizedEffectiveHamiltonian_class(vec_H, res)


# Section 5

## Script S.5.1: Defining Pool Of Operators/Gates <a name="ScriptSpt5pt1"></a>

In [ ]:
from itertools import combinations, product
from qflux.variational_methods.qmad.ansatz import Ansatz_class, PauliOperator

# Define Pauli matrices
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sz = np.array([[1, 0], [0, 0]])

def build_pool(nqbit):
    pauliStr = ["sx", "sz", "sy"]
    res = []
    # Iterate over combinations of qubit indices and Pauli operators
    for order in range(1, 3):
        for idx in combinations(range(1, nqbit + 1), order):
            for op in product(pauliStr, repeat=order):
                res.append(PauliOperator(op, list(idx), 1, nqbit))
    return res

def Ansatz(u0, relrcut, theta=[], ansatz=[]):
    nqbit = int(np.log2(len(u0)))
    pool_qubit = 2 * nqbit
    # Vectorize initial state
    u0 = np.outer(u0, u0).flatten()
    # Build operator pool
    pool = build_pool(pool_qubit)
    return Ansatz_class(nqbit, u0, relrcut, pool, theta, ansatz)


# Section 6

## Script S.6.1: Amplitude Damping Channel <a name="ScriptSpt6pt1"></a>

In [ ]:
import numpy as np
from qflux.variational_methods.qmad.solver import solve_avq_vect
from qflux.variational_methods.qmad.effh import VectorizedEffectiveHamiltonian
from qflux.variational_methods.qmad.ansatzVect import Ansatz
import matplotlib.pyplot as plt

# Execution Code for Amplitude damping channel example
sx = np.array([[0, 1], [1, 0]])
sy = np.array([[0, -1j], [1j, 0]])
sp = (sx + 1j * sy) / 2
Id = np.eye(2, dtype=np.complex128)

tf = 1000e-12
dt = 1000e-13
gamma = 1.52e9
H = np.eye(2, dtype=np.complex128)
lind = sp
H = VectorizedEffectiveHamiltonian(H, gamma, lind)
u0 = np.array([1 / 2, np.sqrt(3) / 2], dtype=np.complex128) #it should be normalized initial state with 2^n length

ansatz = Ansatz(u0, relrcut=1e-6)
res = solve_avq_vect(H, ansatz, [0, tf], dt)


excited = [res.u[i][1, 1].real for i in range(10)]
ground = [res.u[i][0, 0].real for i in range(10)]
times = np.linspace(0, 1000e-12, 10)

qt_times = np.arange(0, tf, 1000e-14)  # time array
plt.plot(qt_times * 1e12, ground_population, label="Ground State (QuTiP)", linewidth=2, color='tab:blue')
plt.plot(qt_times * 1e12, excited_population, label="Excited State (QuTiP)", linewidth=2, color='tab:orange')
plt.plot(times * 1e12, ground, '*', label="Ground State (UAVQDS)", color='tab:blue')
plt.plot(times * 1e12, excited, '*', label="Excited State (UAVQDS)", color='tab:orange')
plt.xlabel("Time (ps)")
plt.ylabel("Amplitude")
plt.legend()
plt.show()


# Section 7

## Script S.7.1: Preprocessing Hamiltonian For SSE Approach <a name="ScriptSpt7pt1"></a>

In [ ]:
class EffectiveHamiltonian_class:
    def __init__(self, He, Ha, Llist, LdL):
        self.He = He  # Hermitian part
        self.Ha = Ha  # Anti-Hermitian part
        self.Llist = Llist  # List of Lindblad operators
        self.LdL = LdL  # List of L†L

def EffectiveHamiltonian( mats, Llist):
    """
    Create an EffectiveHamiltonian object based on provided parameters.
    :param mats: List of matrices (Hamiltonian terms).
    :param Llist: List of lists of Lindblad operators.
    :return: An instance of EffectiveHamiltonian_class.
    """
    He = sum(mats)  # Sum of Hamiltonian terms as Hermitian part
    Ha = 0.0  # Initialize anti-Hermitian part
    LdL = []  # Initialize the list for Lindblad operator products

    for  LL in Llist:
        for L in LL:
            L_dagger_L = (L.conj().T @ L)
            LdL.append(L_dagger_L)  # Append to LdL list
            Ha += L_dagger_L  # Sum for the anti-Hermitian part

    # Return the Effective Hamiltonian object
    return EffectiveHamiltonian_class(He, 0.5 * Ha, [L for LL in Llist for L in LL], LdL)


# Section 8

## Script S.8.1: Parallel Processing Of Trajectories <a name="ScriptSpt8pt1"></a>

In [ ]:
from multiprocessing import Pool
def run_trajectories(num_trajectory, H, ansatz, tf, dt):
    # Create a list of tuples with the required parameters for each trajectory
    param_list = [(H, ansatz, tf, dt) for _ in range(num_trajectory)]

    with Pool() as pool:
        results = pool.starmap(solve_avq_trajectory, param_list)

    return results


# Section 9

## Script S.9.1: Setting Up Parameters For FMO Complex <a name="ScriptSpt9pt1"></a>

In [ ]:
#Hamiltonian
H = [
    [0, 0, 0, 0, 0],
    [0, 0.0267, -0.0129, 0.000632, 0],
    [0, -0.0129, 0.0273, 0.00404, 0],
    [0, 0.000632, 0.00404, 0, 0],
    [0, 0, 0, 0, 0],
]
H_fmo= np.pad(H, ((0, 3), (0, 3)), mode='constant')

alpha, beta, gamma = 3e-3, 5e-7, 6.28e-3

# Define the alpha operators
Llist_f = [(np.diag([0] * i + [np.sqrt(alpha)] + [0] * (4 - i))) for i in range(1, 4)]

# Define the beta operators
Llist_f += [(np.array([[np.sqrt(beta) if i == 0 and j == k else 0 for j in range(5)] for i in range(5)])) for k in range(1, 4)]

# Define the gamma operator
L_temp = np.zeros((5, 5))
L_temp[4, 3] = np.sqrt(gamma)
Llist_f.append(L_temp)
Llist_f_padded = [np.pad(matrix, ((0, 3), (0, 3)), mode='constant') for matrix in Llist_f]

#initial state
u0_fmo = np.zeros(8,dtype=np.complex128)
u0_fmo[1] = 1


## Script S.9.2: Setting Up Parameters For Trajectory Method <a name="ScriptSpt9pt2"></a>

In [ ]:
from qflux.variational_methods.qmad.solver import solve_avq_trajectory
from qflux.variational_methods.qmad.effh import EffectiveHamiltonian
from qflux.variational_methods.qmad.ansatz import Ansatz

# Guard for multiprocessing in Jupyter
if __name__ == "__main__":
    # Define your parameters (these are placeholders)
    tf = 450          # Final time
    dt = 5            # Time step
    num_trajectory = 200  # Number of trajectories
    H = EffectiveHamiltonian([H_fmo], [Llist_f_padded])  # Initialize the effective Hamiltonian
    ansatz = Ansatz(u0_fmo, relrcut=1e-5)  # Create an Ansatz instance

    # Running the parallel trajectories
    results = run_trajectories(num_trajectory, H, ansatz, tf, dt)



## Script S.9.3: Plot For FMO Complex Simulation Output <a name="ScriptSpt9pt3"></a>

In [ ]:
import numpy as np

# Define the observables as diagonal matrices for expectation value calculations
Mexp_f = [
    np.diag([0, 1, 0, 0, 0, 0, 0, 0]),  # Observable 1
    np.diag([0, 0, 1, 0, 0, 0, 0, 0]),  # Observable 2
    np.diag([0, 0, 0, 1, 0, 0, 0, 0]),  # Observable 3
    np.diag([1, 0, 0, 0, 0, 0, 0, 0]),  # Observable 4
    np.diag([0, 0, 0, 0, 1, 0, 0, 0])   # Observable 5
]

# Initialize a list to store the average expectation values
average_expectation_values = []

# Loop over each trajectory to accumulate expectation values
for j in range(num_trajectory):
    # Loop over each observable defined in Mexp_f
    for k, observable in enumerate(Mexp_f):
        expectation_values = []  # List to hold expectation values for the current observable

        # Calculate expectation values for the current trajectory
        for i, psi in enumerate(results[j].psi):
            psi_dagger = np.conjugate(psi).T  # Conjugate transpose of the wave function
            rho = np.outer(psi, psi_dagger)    # Calculate the density matrix
            expectation_value = np.trace(np.dot(rho, observable))  # Compute the expectation value

            # Store the real part of the expectation value
            expectation_values.append(expectation_value.real)

        # Accumulate expectation values for averaging later
        if len(average_expectation_values) <= k:
            average_expectation_values.append(np.array(expectation_values))  # Initialize if not already done
        else:
            average_expectation_values[k] += np.array(expectation_values)  # Sum the values for this observable

# Convert time to femtoseconds for plotting (assuming results[0].t contains time data)
results_t_converted = [t for t in results[0].t]

# Average the accumulated expectation values over all trajectories
average_expectation_values = [ev / num_trajectory for ev in average_expectation_values]


In [ ]:
# Time evolution
qutip_times = np.linspace(0.0, 450.0, 2000)

# population

# Plotting the results
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
labels = ["State 1", "State 2", "State 3", "Ground State", "Sink State"]
colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"]
counter = 0
for expec, label in zip(population, labels):
    ax.plot(qutip_times, expec, label=label, color=colors[counter])
    counter += 1

sse_labels = ["State 1 (SSE)", "State 2 (SSE)", "State 3 (SSE)", "Ground State (SSE)", "Sink State (SSE)"]
counter = 0
for expec, label in zip(average_expectation_values, sse_labels):
    ax.plot(results_t_converted, expec, '*', label=label, color=colors[counter])
    counter += 1
ax.set_xlabel('Time(fs)')
ax.set_ylabel('Population')
ax.legend(loc='upper right', ncol=2)
plt.tight_layout()
plt.show()